In [8]:
from classes.SAMMed2DClass import SAMMed2DInferer
from utils.base_classes import Points
import utils.analysis as anUt
import utils.prompt_old as prUt
from utils.image import read_im_gt
from utils.interactivity import iterate_2d

import numpy as np

# Obtain model
device = 'cuda'
sammed2d_checkpoint_path = "/home/t722s/Desktop/UniversalModels/TrainedModels/sam-med2d_b.pth"
sammed2d_inferer = SAMMed2DInferer(sammed2d_checkpoint_path, device)

# Obtain image, gt
img_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/imagesTr/BDMAP_00000001_0000.nii.gz'
gt_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/labelsTr/BDMAP_00000001.nii.gz'
img, gt = read_im_gt(img_path, gt_path, 3)


True
*******load /home/t722s/Desktop/UniversalModels/TrainedModels/sam-med2d_b.pth


In [6]:
# Experiment: n randomly sampled points from foreground
seed = 11121
# n_clicks = 1
n_clicks = 5
point_prompt = prUt.get_pos_clicks2D_row_major(gt, n_clicks, seed = seed)
segmentation = sammed2d_inferer.predict(img, point_prompt, use_stored_embeddings=True)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 141.16it/s]


0.9142033388619348

In [7]:
# Experiment: 2d bounding box per slice with foreground
box_prompt = prUt.get_minimal_boxes_row_major(gt, 0, 0)
segmentation = sammed2d_inferer.predict(img, box_prompt, use_stored_embeddings=True)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 154.11it/s]


0.9479552096105959

In [8]:
# Experiment: get 3d bbox and slice it to feed it in 2d axially
box_prompt = prUt.get_bbox3d_sliced(gt)

segmentation = sammed2d_inferer.predict(img, box_prompt, use_stored_embeddings=True)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 156.56it/s]


0.7618218132404309

In [9]:
# Experiment: line interpolation
n_slices = 5
interpolation = 'linear'
point_prompt = prUt.line_interpolation(gt, n_slices, interpolation)

segmentation = sammed2d_inferer.predict(img, point_prompt, use_stored_embeddings=True)
anUt.compute_dice(segmentation, gt)

Performing inference on slices: 100%|██████████| 12/12 [00:00<00:00, 25.32it/s]


0.18578502906629027

In [19]:
# Iteratively improve previous segmentation (from line interpolation)
n_slices = 5
interpolation = 'linear'
point_prompt = prUt.line_interpolation(gt, n_slices, interpolation)

segmentation, low_res_logits = sammed2d_inferer.predict(img, point_prompt, use_stored_embeddings=True, return_low_res_logits=True)


initial_prompt = prompt
condition = 'dof'
dof_bound = 90
perf_bound = 0.85
dice_scores, dof, segmentations, prompts = iterate_2d(sammed2d_inferer, img, gt, segmentation, low_res_logits, initial_prompt, sammed2d_inferer.pass_prev_prompts,use_stored_embeddings=True,
                                                      scribble_length = 0.6, contour_distance = 3, disk_size_range= (0,3),
                                                      init_dof = 5, perf_bound = 0.85, dof_bound = dof_bound, seed = seed, detailed = True)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 159.15it/s]


0.9110504232455452
0.9177664653951643
0.9175018545115927
0.9175982870460488
0.9173824539625599
0.9175865225763682
0.9193648569016866
0.919572454879972
0.9198370503348754


In [10]:
# Experiment: box interpolation
n_boxes = 5

seed_boxes = prUt.get_seed_boxes(gt, n_boxes)
box_prompt = prUt.box_interpolation(seed_boxes)
segmentation = sammed2d_inferer.predict(img, box_prompt, use_stored_embeddings=True)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 151.89it/s]


0.946361076722504

In [12]:
# Experiment: Point propagation
seed = 11121
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.point_propagation(sammed2d_inferer, img, seed_point, slices_to_infer, seed, n_clicks, use_stored_embeddings=True)
print(anUt.compute_dice(gt,segmentation))


Propagating up: 100%|██████████| 22/22 [00:00<00:00, 68.94it/s]


0.40216919918882194


In [18]:
# Iteratively improve previous segmentation (from point propagation)
seed = 1
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, low_res_logits, prompt = prUt.point_propagation(sammed2d_inferer, img, seed_point, slices_to_infer, seed, n_clicks, use_stored_embeddings=True, return_low_res_logits=True)


initial_prompt = prompt
condition = 'dof'
dof_bound = 90
perf_bound = 0.85
dice_scores, dof, segmentations, prompts = iterate_2d(sammed2d_inferer, img, gt, segmentation, low_res_logits, initial_prompt, sammed2d_inferer.pass_prev_prompts,use_stored_embeddings=True,
                                                      scribble_length = 0.6, contour_distance = 3, disk_size_range= (0,3),
                                                      init_dof = 5, perf_bound = 0.85, dof_bound = dof_bound, seed = seed, detailed = True)

Propagating up: 100%|██████████| 22/22 [00:00<00:00, 61.63it/s]


0.8213665466281155
0.8205091864261047
0.8198578881518378
0.8213837252830045
0.8215174453603068
0.8212659812587731
0.8218388895479891
0.8322763716696185
0.8321815496121933
0.8332855449783997
0.8327022737268851
0.833809644222007


In [13]:
# Experiment: Box propagation

seed_box = prUt.get_seed_box(gt)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.box_propagation(sammed2d_inferer, img, seed_box, slices_to_infer, use_stored_embeddings=True)

print(anUt.compute_dice(gt,segmentation))

Propagating up: 100%|██████████| 22/22 [00:00<00:00, 53.42it/s]


0.9401919895492582
